In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#logits shape:[bs, nc] 分类层之前的shape
batchsize = 2
num_class = 4

logits = torch.randn(batchsize, num_class) #input unnormalized score
target_indices = torch.randint(num_class, size = (batchsize,))#delta目标分别

target_logits = torch.randn(batchsize, num_class) #非delta目标分布


#方法一调用CE LOSS
ce_loss_fn = torch.nn.CrossEntropyLoss()
ce_loss = ce_loss_fn(logits, target_indices)
print(f"cross entropy loss1: {ce_loss}")

#方法二 target传入的是一个概率分布
ce_loss = ce_loss_fn(logits, torch.softmax(target_logits,dim = -1))
print(f"cross entropy loss2: {ce_loss}")

cross entropy loss1: 1.775658369064331
cross entropy loss2: 1.5176959037780762


In [8]:
#2调用Negative Log Likelihood loss(NLL Loss)
nll_fn = torch.nn.NLLLoss()
#进行归一化后log，为了让Log稳定，我们需要加入一个极小量,target只能传入整形的索引
nll_loss = nll_fn(torch.log(torch.softmax(logits, dim = -1)+1e-7), target_indices) 
print(f"negative log_likelihood loss: {nll_loss}")



negative log_likelihood loss: 1.7756574153900146


In [9]:
#3. klloss
kld_loss_fn = torch.nn.KLDivLoss()
kld_loss = kld_loss_fn\
    (torch.log(torch.softmax(logits, dim = -1)), torch.softmax(target_logits,dim = -1))

print(f'Kullback–Leibler divergence loss:{kld_loss}')


Kullback–Leibler divergence loss:0.09055455774068832


d:\Anaconda\lib\site-packages\torch\nn\functional.py:2904: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


In [13]:
#4.验证 CE = IE + KLD
#不做batch，对每个样本单独返回交叉熵
ce_loss_fn_sample = torch.nn.CrossEntropyLoss(reduction = "none")
ce_loss_sample = ce_loss_fn_sample(logits, torch.softmax(target_logits,dim = -1))
print(f"cross entropy loss sample: {ce_loss_sample}")

#计算KL散度
kld_loss_fn_sample = torch.nn.KLDivLoss(reduction="none")
kld_loss_sample = kld_loss_fn_sample\
    (torch.log(torch.softmax(logits, dim = -1)), torch.softmax(target_logits,dim = -1)).sum(-1)
print(f'Kullback–Leibler divergence loss sample:{kld_loss_sample}')

#信息熵
target_information_entropy = torch.distributions.Categorical\
    (probs = torch.softmax(target_logits,dim = -1)).entropy()
print(f'information entropy loss sample:{target_information_entropy}')

print(torch.allclose(ce_loss_sample,kld_loss_sample + target_information_entropy))


cross entropy loss sample: tensor([1.2287, 1.8067])
Kullback–Leibler divergence loss sample:tensor([0.2415, 0.4829])
information entropy loss sample:tensor([0.9872, 1.3237])
True


In [23]:
# 5.调用binary cross entropy loss
batchsize = 2
bce_loss_fn = torch.nn.BCELoss()
logits = torch.randn(batchsize)
prob_1 = torch.sigmoid(logits)
target = torch.randint(2, size = (batchsize, )) #target只能是0，1
bce_loss = bce_loss_fn(prob_1, target.float())
print(prob_1.shape)
print(target)
print(f'bce_loss:{bce_loss}')
#bce的一般形式就是nll loss，用nll loss算出来是一样的，用Nll loss代替
prob_0 = 1 - prob_1.unsqueeze(-1)
prob = torch.cat([prob_0, prob_1.unsqueeze(-1)], dim=-1)
nll_loss_binary = nll_fn(torch.log(prob), target)
print(f'nll_loss_binary:{nll_loss_binary}')


torch.Size([2])
tensor([1, 0])
bce_loss:0.547231137752533
nll_loss_binary:0.547231137752533


In [3]:
#6.余弦相似度cosine similarity loss
import torch
batchsize = 2
consine_loss_fn = torch.nn.CosineEmbeddingLoss()
v1 = torch.randn(batchsize, 512)
v2 = torch.randn(batchsize, 512)
target = torch.randint(2, size = (batchsize,)) * 2 -1 #(0,1) -> (0,2) -> (-1,1)
consine_loss = consine_loss_fn(v1,v2,target)
print(f'consine_loss:{consine_loss}')

consine_loss:0.944987416267395
